# Qiskit Dell Runtime Examples

## Local Execution
    
The following program walks through a (simple) example usage of the 
Qiskit Dell Runtime in a local execution environment: i.e. potentially
using a locally installed simulator or a remote call directly from a
local machine to a remote simulator or QPU.

In [ ]:
from dell_runtime import DellRuntimeProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os


If the program that interacts with the simulator/QPU is small enough,
it can be stored as a string in the file that interfaces with the 
provider. Both directories and files can be taken as input, as well.

In [ ]:
RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results using either simulator or QPU backend.
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

The DellRuntimeProvider is an interface that offers a choice of runtime (local or remote). The client is able to select through this interface whether or not they would like to run their code on a remote environment

In [ ]:
provider = DellRuntimeProvider()

The runtime is a service provided that allows clients to upload, update,
view, and run programs inside an execution environment. Since the client
has not specified a remote runtime to the provider it defaults to local.

In [ ]:
program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {program_id}")
programs = provider.runtime.pprint_programs(refresh=True)

The following updates the existing program with a new description - this can be done for any of the metadata fields or the program data itself, though changes to the program data are not shown in the `pprint_programs` output.

In [ ]:
provider.runtime.update_program(program_id, description="IBM/Dell Updated Qiskit Runtime Program")
programs = provider.runtime.pprint_programs(refresh=True)

Below we use the Qiskit QuantumCircuit to create a circuit for our program to run. We then place that circuit in `program_inputs` - a dictionary of things that will be provided to our runtime program.

In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

Through the `provider` we are able to run an instance of our program with the inputs we have created. 

When we run a job locally, a new process is started. This new process returns results to the main process via a socket connection.

In [ ]:
job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

We can obtain a job's final results and specify a timeout for how long we are willing to wait. If no timeout is specified, the function will return `None` or the final results if they are present.

In [ ]:
results = job.result(timeout=60)
print(results)

We can also provide a callback function to the runtime for a job. A thread launched in the client process to poll for messages will call the callback when a non-final message is received.

In [ ]:
def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')

job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

You may also specify a different backend on which you would like the quantum code to run. The default backend is the Qiskit Aer simulator.

In [ ]:
program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

## Remote Execution

The following example does mainly the same things as the local version, but establishes a connection to a remote server on which to run bundled code. The program starts identically to the local example:

In [ ]:
from dell_runtime import DellRuntimeProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os

RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

Here we get our first difference - the `provider.remote()` call establishes a connection to our remote sever running on Kubernetes. 

If SSO is not enabled on the server, the client is returned a user ID that they may save and set as an environment variable (`$QDR_ID`) to return to uploaded data.

If SSO is enabled on the server, the client will follow the usual set of SSO authentication steps (logging in using a pop-up browser window with their credentials) and the server will authenticate them using a token they send back.

In [ ]:
provider = DellRuntimeProvider()
provider.remote(REMOTE_RUNTIME)

Uploading a program looks exactly the same as it did in the local version.

In [ ]:
text_program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {text_program_id}")

Printing out program metadata and updating programs also works the same as locally.

In [ ]:
programs = provider.runtime.pprint_programs(refresh=True)
provider.runtime.update_program(text_program_id, description="IBM/Dell Updated Qiskit Runtime Program")
programs = provider.runtime.pprint_programs(refresh=True)

It is also possible to upload programs stored in files or directories. To do so, instead of providing a string containing the entire program to `provider.runtime.run()` you may provide a path to a file or directory:

In [ ]:
file_program_id = provider.runtime.upload_program("qka.py", description="File Upload to Orchestrator")
dir_program_id = provider.runtime.upload_program("./qkad", description="Directory Upload to Orchestrator")

You'll be able to see those programs uploaded when you print out the list:

In [ ]:
provider.runtime.pprint_programs(refresh=True)

From here we'll do the same things that we did in the local version. Set up a circuit, pass it as input to an instance of our circuit runner program, and then obtain our results:

In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

job = provider.runtime.run(text_program_id, options=None, inputs=program_inputs)

results = job.result(timeout=60)
print(results)

We can also do the same callback feature we saw locally and run on a backend provided on the remote server:

In [ ]:
program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(text_program_id, inputs=program_inputs, options=None, callback=callback_function)
results = job.result(timeout=600)
print(results)

## Common Algorithms

### QKA

It is wholly possible to run a Quantum Kernel Alignment implementation on the Qiskit Dell Runtime. Below is an example that utilizes the directory upload feature (the bundle uploaded is located in `../examples/programs/qkad`) to execute an instance of QKA.

The inputs for this version are already part of the bundle, though it is possible to manipulate the files so that inputs are generated as part of the client code and provided to the bundle upon initiating a job.

In [ ]:
from dell_runtime import DellRuntimeProvider
from qiskit import QuantumCircuit
import pandas as pd
from time import sleep
import os
import base64
import shutil
import json

provider = DellRuntimeProvider()

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

provider.remote(os.getenv("SERVER_URL"))
here = os.path.dirname(os.path.realpath(__file__))

program_id = provider.runtime.upload_program(here + "../examples/programs/qkad", metadata=RUNTIME_PROGRAM_METADATA)

job = provider.runtime.run(program_id, options=None, inputs={})

res = job.result(timeout=1000)
print(res)

### VQE

It is also possible to run Variational Quantum Eigensolver algorithms using the Qiskit Dell Runtime. An example of the client code is visible below (adapted from the IBM Qiskit Textbook):

In [ ]:
from qiskit.opflow import Z, I
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit.algorithms.optimizers import SPSA
from dell_runtime import DellRuntimeProvider
import os
from time import sleep
from datetime import datetime, timedelta

num_qubits = 4
hamiltonian = (Z ^ Z) ^ (I ^ (num_qubits - 2))
target_energy = -1


# the rotation gates are chosen randomly, so we set a seed for reproducibility
ansatz = EfficientSU2(num_qubits, reps=1, entanglement='linear', insert_barriers=True)
# ansatz.draw('mpl', style='iqx')

optimizer = SPSA(maxiter=50)

np.random.seed(10)  # seed for reproducibility
initial_point = np.random.random(ansatz.num_parameters)
intermediate_info = {
    'nfev': [],
    'parameters': [],
    'energy': [],
    'stddev': []
}

timestamps = []

def raw_callback(*args):
    (nfev, parameters, energy, stddev) = args[0]
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    
vqe_inputs = {
    'ansatz': ansatz,
    'operator': hamiltonian,
    'optimizer': {'name': 'SPSA', 'maxiter': 15},  # let's only do a few iterations!
    'initial_point': initial_point,
    'measurement_error_mitigation': True,
    'shots': 1024,
}

provider = DellRuntimeProvider()
provider.remote(os.getenv("SERVER_URL"))
program_id = provider.runtime.upload_program("vqe.py", description="Variational Quantum Eigensolver Program")

job = provider.runtime.run(
    program_id=program_id,
    inputs=vqe_inputs,
    options=None,
    callback=raw_callback
)

print('Job ID:', job.job_id)

result = job.result()
while not result:
    print('no result yet.')
    sleep(0.5)
    result = job.result()

print(f"Intermediate Results: {intermediate_info}")
print(f'Reached {result["optimal_value"]} after {result["optimizer_evals"]} evaluations.')
print('Available keys:', list(result.keys()))